# Movie Recommender System - Creation Recommender Systems

### Content of this notebook

1. General recommendations
2. Recommendations based on content
3. Recommendations based on collaborative filtering
<br>
    3.1 Item-item memory based
    <br>
    3.2 Model based

*For the preparation and exploration of the data, please see the notebook named 'Data_Preparation_and_Exploration'*

Importing the libraries used in this notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unidecode
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, SVDpp, KNNWithMeans, NMF, SlopeOne, Dataset, Reader, accuracy
from surprise.model_selection import GridSearchCV, cross_validate
from surprise.model_selection.split import KFold
from sklearn.model_selection import train_test_split
import random
from collections import defaultdict
import itertools

Unpickle the dataframes prepared in the previous notebook.

In [2]:
with open('pickles/df_small.pkl', 'rb') as f_dfsmall:
    df_small = pickle.load(f_dfsmall)
    
with open('pickles/df_expand_genres.pkl', 'rb') as f_dfgenres:
    df_expand_genres = pickle.load(f_dfgenres)
    
with open('pickles/genres.pkl', 'rb') as f_genres:
    genres = pickle.load(f_genres)
    
with open('pickles/ratings.pkl', 'rb') as f_ratings:
    ratings = pickle.load(f_ratings)

## 1. General recommendations

First, I will prepare a simple function that will recommend movies based on their weighted vote average, with the option to filter by genre and original language. This is a very general way of recommending movies, without any personalization other than the provided filters.

In [3]:
def general_recommendation(genre=None, original_language=None, quantity=10, ):
    
    '''Function that will recommend movies based on their weighted vote average,
    with the option to filter by genre and original language.
    
    Parameters
    ----------------
    - genre: str
    - original_language: str
    - quantity: int, quantity of recommendations to return
    
    '''
    df_small_sorted = df_small.sort_values('weighted_vote_average', ascending=False)
    df_expand_genres_sorted = df_expand_genres.sort_values('weighted_vote_average', ascending=False)
    
    if genre == None:
        if original_language == None:
            df = df_small_sorted[['title','weighted_vote_average']].head(quantity)
        else:
            df = df_small_sorted.loc[(df_small_sorted.original_language == original_language)]
            df = df[['title','weighted_vote_average']].head(quantity)
    
    else:
        if original_language == None:                                     
            df = df_expand_genres_sorted.loc[(df_expand_genres_sorted.genres_list == genre)]
            df = df[['title','weighted_vote_average']].drop_duplicates().head(quantity)
        else:                                         
            df = df_expand_genres_sorted.loc[(df_expand_genres_sorted.original_language == original_language)&\
                                      (df_expand_genres_sorted.genres_list == genre)]
            df = df[['title','weighted_vote_average']].drop_duplicates().head(quantity)
    
    df.index = np.arange(1, len(df) + 1)
    
    return df

In [4]:
general_recommendation(genre='Action', original_language='en')

,title,weighted_vote_average
1,The Dark Knight,8.243606
2,The Empire Strikes Back,8.093833
3,Inception,8.055753
4,The Lord of the Rings: The Return of the King,8.025639
5,Star Wars,8.010571
6,The Lord of the Rings: The Fellowship of the Ring,7.934962
7,The Lord of the Rings: The Two Towers,7.924803
8,Guardians of the Galaxy,7.845535
9,The Matrix,7.840144
10,Scarface 1983,7.822081


## 2. Recommendations based on content

In order to make a recommendation based on characteristics of the movies, I will generate a large string that contains the genres, main actors, directors and production companies of each movie. After, I will use the cosine similarity to calculate the similarity between movies.

First, I will create a copy of the original dataframe containing only the columns that I will use for the content based recommendation.

In [5]:
df2 = df_small[['title','movieId','genres_list','cast_top3','director_list', 'prodcompany_list', 'keywords_list']].copy()

I will create a small function that will convert lists to strings. In case a list item consists of multiple words, these words are joined together. This is to avoid similarities between movies that shouldn't be there. For example, movies with the actors Tom Hanks and Tom Cruise, will both have the word 'Tom' in common. This does not mean though that these movies are 'similar'. 

In [6]:
def list_to_string(x):
    listado = [str.lower(i.replace(" ", "")) for i in x]
    return ' '.join(listado)

Applying the 'list_to_string' function on the features of the dataframe.

In [7]:
features = ['genres_list','cast_top3','director_list','prodcompany_list','keywords_list']

for feature in features:
    df2[feature] = df2[feature].apply(list_to_string)

Creating a new column that combines the different strings with features into one large string and clean this string.

In [8]:
df2['features_combined'] = df2.genres_list.str.cat([df2.cast_top3, df2.director_list, df2.prodcompany_list], sep=' ')
df2['features_combined'] = df2['features_combined'].apply(lambda x: unidecode.unidecode(x))
df2['features_combined'] = df2['features_combined'].apply(lambda x: re.sub(r'([^\s\w]|_)+','',x))

In [9]:
df2.features_combined[0]

'animation comedy family tomhanks timallen donrickles johnlasseter pixaranimationstudios'

Vectorizing the features.

In [10]:
vectorizer = CountVectorizer().fit(df2['features_combined'])
features_vectorized = vectorizer.transform(df2['features_combined'])
features_vectorized = features_vectorized.astype(np.float32)

In [11]:
features_vectorized.shape

(9010, 20924)

In [12]:
with open('pickles/features_vectorized.pkl', 'wb') as f_vecfeatures:
    pickle.dump(features_vectorized, f_vecfeatures) 

Using the cosine similarity to calculate the similarity between the features of the different movies.

In [13]:
csim_features = cosine_similarity(features_vectorized)
csim_features = pd.DataFrame(csim_features, columns=df2.title, index=df2.title)

In [14]:
csim_features.shape

(9010, 9010)

Creating a function that will recommend movies that are similar to the input movie based on the following features: genres, main actors, directors and production companies.

In [15]:
def recommendation_content(title, quantity=10):
    
    '''Function that takes an input movie and recommends movies with similar features as the input movie.
    
    Parameters
    ----------------
    - title: str, title of the input movie
    - quantity: int, quantity of recommendations to return
    
    '''
            
    df_similar_features = pd.DataFrame(csim_features.loc[title].sort_values(ascending=False)).reset_index()
    df_similar_features.columns = ['title', 'cosine_sim']
    df_similar_features = df_similar_features.merge(df_small[['title', 'weighted_vote_average']], on='title')
    df_similar_features = df_similar_features.drop(df_similar_features[df_similar_features.title == title].index)
    df_similar_features = df_similar_features.drop(df_similar_features[df_similar_features.weighted_vote_average<7].index)
        
    return df_similar_features.head(quantity)

In [16]:
%%time
recommendation_content('The Dark Knight')

Wall time: 76.4 ms


,title,cosine_sim,weighted_vote_average
1,The Dark Knight Rises,0.880705,7.552751
2,Batman Begins,0.846154,7.446915
3,The Prestige,0.613941,7.876536
4,Inception,0.480384,8.055753
13,Heat,0.418121,7.481785
19,Interstellar,0.400320,8.044694
37,Training Day,0.384615,7.130469
49,Scarface 1983,0.369800,7.822081
52,Dirty Harry,0.369800,7.017779
59,M,0.369800,7.277003


## 3. Recommendations based on collaborative filtering

To create a recommender system based on collaborative filtering models, I will use the Surprise library (a Python scikit for building and analyzing recommendation systems dealing with explicit rating data).

Documentation: http://surpriselib.com/

Instead of basing the recommendations on the movies characteristics, these recommendations wll be solely based on users ratings. The ratings dataframe contains ratings with a scale from 0 to 5 given by 671 users. For collaborative filtering models, a bigger dataset with more different users would be ideal. However, due to computational limitations, I will have to work with this smaller subset of 671 users.

I will try out several types of models and afterwards choose the one that works best for my dataset:

1. **Item-item memory based**: 
    - **KNNwithMeans**. KNN does not make any assumptions about the distribution of the data, it is based only on the similarity of the features. KNNwithMeans takes into account the average of the ratings each user gives to correct for their level of optimism.
<br>
<br>
2. **Model based**:
   - **Slope One** algorithm
   - **Matrix Factorization**: I will try out both the **SVD** and **NMF** algorithm.

The evaluation metrics used in this section:

- **RMSE**: Root Mean Squared Error. This metric will tell me how well my model predicts the users ratings.
- **FCP**: Fraction of Concordant Pairs, the proportion of item pairs that were correctly ranked. This metric will tell me how well my model predicts the order of users preferences.
- **Precision@K**: The proportion of items recommended in the top-K that are relevant (rating>=3.5). This will tell me how relevant the recommended items are to the user.
- **Recall@K**: The proportion of relevant elements (rating>=3.5) found in the top-K recommendations. This will tell me what percentage of the relevant items are recommended by my model.
- **Coverage**: The percentage of items included in all the recommendations over the number of potential items. This metric will tell me how diverse my recommendations are.

In [17]:
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


Splitting the dataset into a train and test set. I will use 80% of the data for the train set and 20% for the test set.

In [18]:
data_train_original, data_test_original = train_test_split(ratings, test_size=0.2, random_state=4)

In [19]:
data_train_original.shape

(80003, 3)

In [20]:
data_test_original.shape

(20001, 3)

In [21]:
kf = KFold(n_splits=3, random_state=7)

The models in the Surprise library requiere the input data to have a specific format. Below I will use several functions of the Surprise library to prepare the input data. 

In [22]:
reader_train = Reader(rating_scale=(data_train_original["rating"].min(),data_train_original["rating"].max()))
data_train = Dataset.load_from_df(data_train_original,reader_train)

reader_test = Reader(rating_scale=(data_test_original["rating"].min(),data_test_original["rating"].max()))
data_test = Dataset.load_from_df(data_test_original,reader_test)

reader_full = Reader(rating_scale=(ratings["rating"].min(),ratings["rating"].max()))
data_full = Dataset.load_from_df(ratings,reader_full)

trainset = data_train.build_full_trainset()

testset = data_test.build_full_trainset()
testset = testset.build_testset()

anti_testset = data_test.build_full_trainset()
anti_testset = anti_testset.build_anti_testset()

I will  create a function to calculate the following evaluation metrics: Precision@K, Recall@K and Coverage. The evaluation metrics RMSE and FCP are included in the Surprise library, and have already been imported at the beginning of this notebook.

In [23]:
# Code source: https://surprise.readthedocs.io/en/stable/FAQ.html

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [24]:
# Code source: https://surprise.readthedocs.io/en/stable/FAQ.html

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [25]:
def coverage(predictions, n=10, dataset_original=data_test_original):
    '''Returns the percentage of movies included in the different recommendation lists
    (top-n) over the total number of movies.'''

    top_n = get_top_n(predictions, n=n)

    recommendations = []

    for uid, user_ratings in top_n.items():
        for (iid, _) in user_ratings:
            recommendations.append(iid)
        
    unique_movies_recommended = len(set(recommendations))
    unique_movies_testset = dataset_original.movieId.nunique()

    coverage = unique_movies_recommended/unique_movies_testset
    
    return coverage

### 3.1 Item-Item Memory Based

First, I will try out the **KNNwithMeans** algorithm.
<br>
<br>
The gridsearch of Surprise allows to optimize for RMSE, MSE, MAE, and FCP. I will use the best FCP hyperparameters, because the most important thing for me is to rank the movies in the correct order. I have commented out the gridsearch section, because it is computationally heavy, but I am using the best hyperparameters obtained by this gridsearch earlier.

In [26]:
#param_grid = {'min_k': [20, 50],
#              'sim_options': {'name': ['msd', 'cosine','pearson'],
#                              'min_support': [1, 5],
#                              'user_based': [False]}
#              }

In [27]:
#grid_knn = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse','fcp'], refit=True, cv=kf)

In [28]:
#%%time
#grid_knn.fit(data_train)

In [29]:
#grid_knn.best_score

In [30]:
#grid_knn.best_params

I will train the KNNWithMeans algorithm with the trainset and the hyperparameters obtained in the gridsearch.

In [31]:
sim_options = {"name": "msd", "min_support":1, "user_based": False}
k = 50

In [32]:
%%time
model_KNN = KNNWithMeans(sim_options=sim_options, min_k=k)
model_KNN.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Wall time: 10.4 s


I will predict the recommendations based on the testset, and calculate the RMSE, FCP, Precision@K and Recall@K.
<br>
<br>
*Note: Due to computational limitations of my laptop, I cannot calculate the coverage metric for KNNWithMeans. KNNWithMeans is a memory based algorithm. For the coverage metric, I would need to calculate the recommendations for all users taking into account all the movies in the testset. My computer does not have enough memory to do so. The other metrics only take into account the movies evaluated by the user, and not all movies in the dataset, and are therefore lighter to calculate.*

In [33]:
pred_KNN_test = model_KNN.test(testset)

RMSE = accuracy.rmse(pred_KNN_test)
FCP = accuracy.fcp(pred_KNN_test)

precisions, recalls = precision_recall_at_k(pred_KNN_test)

Precision = sum(prec for prec in precisions.values()) / len(precisions)
Recall = sum(rec for rec in recalls.values()) / len(recalls)

RMSE: 1.0019
FCP:  0.6488


I will add the results of the evaluation metrics to a dateframe that I will use to compare the different algorithms.

In [34]:
results = pd.DataFrame(columns=[['model','RMSE', 'FCP', 'Precision@K', 'Recall@K']])

In [35]:
results.loc[0, 'model'] = 'KNNWithMeans'
results.loc[0, 'RMSE'] = RMSE
results.loc[0, 'FCP'] = FCP
results.loc[0, 'Precision@K'] = Precision
results.loc[0, 'Recall@K'] = Recall

In [36]:
results

,model,RMSE,FCP,Precision@K,Recall@K
0,KNNWithMeans,1.00185,0.648809,0.777222,0.554911


### 3.2 Model-based

#### Slope One

Second, I will try out the Slope One algorithm. This algorithm does not have any hyperparameters to optimize.

In [37]:
model_SlopeOne = SlopeOne()

In [38]:
%%time
model_SlopeOne.fit(trainset)

Wall time: 5.57 s


I will predict the recommendations based on the testset, and calculate the RMSE, FCP, Precision@K, Recall@K and Coverage.

In [39]:
pred_SlopeOne_test = model_SlopeOne.test(testset)

RMSE = accuracy.rmse(pred_SlopeOne_test)
FCP = accuracy.fcp(pred_SlopeOne_test)

precisions, recalls = precision_recall_at_k(pred_SlopeOne_test)

Precision = sum(prec for prec in precisions.values()) / len(precisions)
Recall = sum(rec for rec in recalls.values()) / len(recalls)

RMSE: 0.9305
FCP:  0.6421


In [40]:
pred_SlopeOne_antitest = model_SlopeOne.test(anti_testset)

In [41]:
Coverage = coverage(pred_SlopeOne_antitest)

In [42]:
results.loc[1, 'model'] = 'Slope One'
results.loc[1, 'RMSE'] = RMSE
results.loc[1, 'FCP'] = FCP
results.loc[1, 'Precision@K'] = Precision
results.loc[1, 'Recall@K'] = Recall
results.loc[1, 'Coverage'] = Coverage

In [43]:
results

,model,RMSE,FCP,Precision@K,Recall@K,Coverage
0,KNNWithMeans,1.00185,0.648809,0.777222,0.554911,NaN
1,Slope One,0.930466,0.642111,0.782997,0.527836,0.081767


#### Matrix Factorization - SVD

Third, I will try out the **SVD** algorithm.
<br>
<br>
The gridsearch of Surprise allows to optimize for RMSE, MSE, MAE, and FCP. I will use the best FCP hyperparameters, because the most important thing for me is to rank the movies in the correct order. I have commented out the gridsearch section, because it is computationally heavy, but I am using the best hyperparameters obtained by this gridsearch earlier.

In [44]:
#param_grid = {'n_epochs': [20, 30, 50],
#              'biased': [True, False],
#              'lr_all': [0.005, 0.01, 0.1],
#              'reg_all': [0.05, 0.1, 0.3]
#              }

In [45]:
#grid_SVD = GridSearchCV(SVD, param_grid, cv=kf, n_jobs=-1, refit=True, measures = ["rmse","fcp"])

In [46]:
#%%time
#grid_SVD.fit(data_train)

In [47]:
#grid_SVD.best_params

In [48]:
#grid_SVD.best_score

I will train the SVD algorithm with the trainset and the hyperparameters obtained in the gridsearch.

In [49]:
model_SVD = SVD(n_epochs=30, biased=True, lr_all=0.01, reg_all=0.1, random_state=4)

In [50]:
%%time
model_SVD.fit(trainset)

Wall time: 10.6 s


I will predict the recommendations based on the testset, and calculate the RMSE, FCP, Precision@K and Recall@K.

In [51]:
pred_SVD_test = model_SVD.test(testset)

RMSE = accuracy.rmse(pred_SVD_test)
FCP = accuracy.fcp(pred_SVD_test)

precisions, recalls = precision_recall_at_k(pred_SVD_test)

Precision = sum(prec for prec in precisions.values()) / len(precisions)
Recall = sum(rec for rec in recalls.values()) / len(recalls)

RMSE: 0.8791
FCP:  0.6641


In [52]:
pred_SVD_antitest = model_SVD.test(anti_testset)

In [53]:
Coverage = coverage(pred_SVD_antitest)

In [54]:
results.loc[2, 'model'] = 'SVD'
results.loc[2, 'RMSE'] = RMSE
results.loc[2, 'FCP'] = FCP
results.loc[2, 'Precision@K'] = Precision
results.loc[2, 'Recall@K'] = Recall
results.loc[2, 'Coverage'] = Coverage

In [55]:
results

,model,RMSE,FCP,Precision@K,Recall@K,Coverage
0,KNNWithMeans,1.00185,0.648809,0.777222,0.554911,NaN
1,Slope One,0.930466,0.642111,0.782997,0.527836,0.081767
2,SVD,0.879087,0.664126,0.808234,0.539637,0.078670


#### Matrix Factorization - NMF

Finally, I will try out the **NMF** algorithm.

In [56]:
#param_grid = {'n_factors': [10, 15, 20],
#              'n_epochs': [20, 50, 100],
#              'biased': [True, False],
#              'reg_pu': [0.01, 0.06, 0.1],
#              'reg_qi': [0.01, 0.06, 0.1]
#              }

In [57]:
#grid_NMF = GridSearchCV(NMF, param_grid, cv=kf, n_jobs=-1, refit=True, measures = ["rmse","fcp"])

In [58]:
#%%time
#grid_NMF.fit(data_train)

In [59]:
#grid_NMF.best_params

In [60]:
#grid_NMF.best_score

I will train the NMF algorithm with the trainset and the hyperparameters obtained in the gridsearch.

In [61]:
model_NMF = NMF(n_factors=10, n_epochs=100, biased=False, reg_pu=0.1, reg_qi=0.1, random_state=4)

In [62]:
%%time
model_NMF.fit(trainset)

Wall time: 12.9 s


I will predict the recommendations based on the testset, and calculate the RMSE, FCP, Precision@K, Recall@K and Coverage.

In [63]:
pred_NMF_test = model_NMF.test(testset)

RMSE = accuracy.rmse(pred_NMF_test)
FCP = accuracy.fcp(pred_NMF_test)

precisions, recalls = precision_recall_at_k(pred_NMF_test)

Precision = sum(prec for prec in precisions.values()) / len(precisions)
Recall = sum(rec for rec in recalls.values()) / len(recalls)

RMSE: 0.9372
FCP:  0.6504


In [64]:
pred_NMF_antitest = model_NMF.test(anti_testset)

In [65]:
Coverage = coverage(pred_NMF_antitest)

In [66]:
results.loc[3, 'model'] = 'NMF'
results.loc[3, 'RMSE'] = RMSE
results.loc[3, 'FCP'] = FCP
results.loc[3, 'Precision@K'] = Precision
results.loc[3, 'Recall@K'] = Recall
results.loc[3, 'Coverage'] = Coverage

In [67]:
results

,model,RMSE,FCP,Precision@K,Recall@K,Coverage
0,KNNWithMeans,1.00185,0.648809,0.777222,0.554911,NaN
1,Slope One,0.930466,0.642111,0.782997,0.527836,0.081767
2,SVD,0.879087,0.664126,0.808234,0.539637,0.078670
3,NMF,0.937238,0.650421,0.808056,0.492116,0.114598


The SVD algorithm has the best scores for RMSE, FCP and Precision@K. However, it has a low Coverage, meaning that there is little variation between its recommendations. I cannot optimize for Coverage because it is computationally too heavy, but I will try to optimize the SVD algorithm for Precision@K instead of FCP, to see if it changes the results.

I will create two functions that will allow me to optimize the hyperparameters of SVD for Precision@K.

In [68]:
def sample_hyperparameters_SVD():

    while True:
        yield {
            "n_epochs": np.random.randint(10, 50),
            "biased": np.random.choice([False, True]),
            "lr_all": np.random.choice([0.005, 0.01, 0.1]),
            "reg_all": np.random.choice([0.05, 0.1, 0.3]),
        }

def random_search_precision_SVD(data_train, data_test, num_samples=10):

    for hyperparams in itertools.islice(sample_hyperparameters_SVD(), num_samples):
        
        model = SVD(**hyperparams)
        trainset = data_train.build_full_trainset()
        model.fit(trainset)
        
        testset = data_test.build_full_trainset()
        testset = testset.build_testset()
        pred = model.test(testset)
        
        precisions, recalls = precision_recall_at_k(pred)
        score = sum(prec for prec in precisions.values()) / len(precisions)

        yield (score, hyperparams, model)

In [69]:
#(score, hyperparams, model) = max(random_search_precision_SVD(data_train, data_test), key=lambda x: x[0])

In [70]:
#print("Best score {} at {}".format(score, hyperparams))

I will train the SVD model on the trainset with the best hyperparameters obtained in the random search.

In [71]:
model_SVD = SVD(n_epochs=37, biased=True, lr_all=0.01, reg_all=0.1, random_state=4)
model_SVD.fit(trainset)

I will predict the recommendations based on the testset, and calculate the RMSE, FCP, Precision@K, Recall@K and Coverage.

In [72]:
pred_SVD = model_SVD.test(testset)

RMSE = accuracy.rmse(pred_SVD)
FCP = accuracy.fcp(pred_SVD)

precisions, recalls = precision_recall_at_k(pred_SVD)

Precision = sum(prec for prec in precisions.values()) / len(precisions)
Recall = sum(rec for rec in recalls.values()) / len(recalls)

RMSE: 0.8773
FCP:  0.6680


In [73]:
pred_antitest_SVD = model_SVD.test(anti_testset)

In [74]:
Coverage = coverage(pred_antitest_SVD)

In [75]:
print('RMSE:', RMSE)
print('FCP', FCP)
print('Precision@K:', Precision)
print('Recall@K', Recall)
print('Coverage', Coverage)

RMSE: 0.8772617423261785
FCP 0.6680481350683322
Precision@K: 0.8095457237621412
Recall@K 0.5426622859350428
Coverage 0.09436299814164774


In [76]:
results.loc[2, 'model'] = 'SVD'
results.loc[2, 'RMSE'] = RMSE
results.loc[2, 'FCP'] = FCP
results.loc[2, 'Precision@K'] = Precision
results.loc[2, 'Recall@K'] = Recall
results.loc[2, 'Coverage'] = Coverage

In [77]:
results

,model,RMSE,FCP,Precision@K,Recall@K,Coverage
0,KNNWithMeans,1.00185,0.648809,0.777222,0.554911,NaN
1,Slope One,0.930466,0.642111,0.782997,0.527836,0.081767
2,SVD,0.877262,0.668048,0.809546,0.542662,0.094363
3,NMF,0.937238,0.650421,0.808056,0.492116,0.114598


The metrics have improved a bit. However, the coverage of SVD remains low, and lower than the NMF coverage.
<br>
<br>
To have more variation in the recommendations I will use the NMF model for the final movie recommender, which has the highest coverage and the other metrics are also acceptable to me. I will also try to optimize the NFM model for Precision@K to see if it improves its results compared to the FCP-optimized model.

I will create two functions that will allow me to optimize the hyperparameters of NMF for Precision@K.

In [78]:
def sample_hyperparameters_NMF():

    while True:
        yield {
            "n_factors": np.random.randint(10,20),
            "n_epochs": np.random.randint(20, 100),
            "biased": np.random.choice([False, True]),
            "reg_pu": np.random.choice([0.01, 0.05, 0.1]),
            "reg_qi": np.random.choice([0.01, 0.04, 0.1]),
        }

def random_search_precision_NMF(data_train, data_test, num_samples=10):

    for hyperparams in itertools.islice(sample_hyperparameters_NMF(), num_samples):
        
        model = NMF(**hyperparams)
        trainset = data_train.build_full_trainset()
        model.fit(trainset)
        
        testset = data_test.build_full_trainset()
        testset = testset.build_testset()
        pred = model.test(testset)
        
        precisions, recalls = precision_recall_at_k(pred)
        score = sum(prec for prec in precisions.values()) / len(precisions)

        yield (score, hyperparams, model)

In [79]:
#(score, hyperparams, model) = max(random_search_precision_NMF(data_train, data_test), key=lambda x: x[0])

In [80]:
#print("Best score {} at {}".format(score, hyperparams))

I will train the NMF model on the train set with the best hyperparameters obtained in the random search.

In [81]:
model_NMF = NMF(n_factors=11, n_epochs=40, biased=True, reg_pu=0.01, reg_qi=0.1, random_state=4)
model_NMF.fit(trainset)

I will predict the recommendations based on the testset, and calculate the RMSE, FCP, Precision@K, Recall@K and Coverage.

In [82]:
pred_NMF = model_NMF.test(testset)

RMSE = accuracy.rmse(pred_NMF)
FCP = accuracy.fcp(pred_NMF)

precisions, recalls = precision_recall_at_k(pred_NMF)

Precision = sum(prec for prec in precisions.values()) / len(precisions)
Recall = sum(rec for rec in recalls.values()) / len(recalls)

RMSE: 0.9120
FCP:  0.6507


In [83]:
pred_antitest_NMF = model_NMF.test(anti_testset)

In [84]:
Coverage = coverage(pred_antitest_NMF)

In [85]:
print('RMSE:', RMSE)
print('FCP', FCP)
print('Precision@K:', Precision)
print('Recall@K', Recall)
print('Coverage', Coverage)

RMSE: 0.9119941305523163
FCP 0.6507272280086235
Precision@K: 0.8005152807391607
Recall@K 0.5244361095511496
Coverage 0.08920090852777204


Although optimizing for Precision@K improves some metrics of the NMF model, it lowers the coverage. For the final recommender system I will use the hyperparameters obtained when optimizing for FCP.

#### Final model with collaborative filtering - NMF

Finally, I will create a function for the recommender system based on collaborative filtering.

In [86]:
def recommendation_nmf(my_ratings, n=10):
    
    '''This functions takes a list of user ratings as an input and 
    returns a recommendation comparing these ratings with other users 
    ratings in the dataset. It uses an NMF algorithm.'''
    
    
    ratings2 = ratings.append(my_ratings)
    reader = Reader(rating_scale=(ratings2["rating"].min(),ratings2["rating"].max()))
    data = Dataset.load_from_df(ratings2,reader)
    
    model_NMF = NMF(n_factors=10, n_epochs=100, biased=False, reg_pu=0.1, reg_qi=0.1, random_state=4)
    trainset = data.build_full_trainset()
    model_NMF.fit(trainset)
    
    testset = trainset.build_anti_testset()
    predictions = model_NMF.test(testset[-9066:-1])
    
    userid = max(ratings2.userId)
    top_n = []
    
    for uid, iid, true_r, est, _ in predictions:
        if uid==userid:
            top_n.append((iid, est))
        
    top_n.sort(key=lambda x: x[1], reverse=True)
    top_n = pd.DataFrame([x[0] for x in top_n], columns=['movieId'])
    top_n = top_n.merge(df_small[['title', 'movieId', 'weighted_vote_average']], on='movieId')
    top_n = top_n.drop(top_n[top_n.weighted_vote_average<7].index)
    
    return top_n.head(n)

Trying out the model with random input ratings.

In [87]:
my_ratings = []

for x in range(20):
    my_ratings.append({'userId': (max(ratings.userId)+1),\
                        'movieId':ratings.movieId.sample().tolist()[0],\
                        'rating':np.random.randint(1,5)})

In [88]:
%%time
recommendation_nmf(my_ratings)

Wall time: 26 s


,movieId,title,weighted_vote_average
38,89774,Warrior,7.404591
49,73881,3 Idiots,7.365964
58,26903,Whisper of the Heart,7.014008
67,2810,Perfect Blue,7.001023
70,1232,Stalker,7.069812
83,26776,Porco Rosso,7.111218
97,1298,Pink Floyd: The Wall,7.027146
123,115122,What We Do in the Shadows,7.067431
192,103235,The Best Offer,7.247510
215,3386,JFK,7.025248


In [89]:
my_ratings = []

for x in range(20):
    my_ratings.append({'userId': (max(ratings.userId)+1),\
                        'movieId':ratings.movieId.sample().tolist()[0],\
                        'rating':np.random.randint(1,5)})

In [90]:
%%time
recommendation_nmf(my_ratings)

Wall time: 21.7 s


,movieId,title,weighted_vote_average
5,2810,Perfect Blue,7.001023
45,26776,Porco Rosso,7.111218
54,920,Gone with the Wind,7.339137
77,73881,3 Idiots,7.365964
96,1298,Pink Floyd: The Wall,7.027146
113,1230,Annie Hall,7.425597
133,923,Citizen Kane,7.626392
170,2739,The Color Purple,7.010159
180,1204,Lawrence of Arabia,7.372975
188,27773,Oldboy 2003,7.745669


In [91]:
my_ratings = []

for x in range(20):
    my_ratings.append({'userId': (max(ratings.userId)+1),\
                        'movieId':ratings.movieId.sample().tolist()[0],\
                        'rating':np.random.randint(1,5)})

In [92]:
%%time
recommendation_nmf(my_ratings)

Wall time: 21.4 s


,movieId,title,weighted_vote_average
15,71838,Law Abiding Citizen,7.037113
36,98491,Paperman,7.453488
51,96588,Pitch Perfect,7.171301
94,71899,Mary and Max,7.251601
102,26903,Whisper of the Heart,7.014008
128,55442,Persepolis,7.106687
135,95311,Presto,7.185039
150,31658,Howl's Moving Castle,7.920374
173,1172,Cinema Paradiso,7.637723
194,3000,Princess Mononoke,7.919445
